In [1]:
from pathlib import Path
from course_intro_ocr_t1.data import MidvPackage
from tqdm import tqdm
from matplotlib import pyplot as plt
from matplotlib import patches
import numpy as np

import cv2
from  torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
from tqdm.notebook import tqdm
from time import time

In [4]:
gpu_number = 4
device = torch.device(device=f'cuda:{gpu_number}')
device

device(type='cuda', index=4)

In [17]:
DATASET_PATH = Path().absolute().parent.parent.parent /'Alexander'/'ocr' /'midv500_compressed'
assert DATASET_PATH.exists(), DATASET_PATH.absolute()
DATASET_PATH

# DATASET_PATH = Path().absolute().parent.parent / 'midv500' / 'midv500_compressed'
# assert DATASET_PATH.exists(), DATASET_PATH.absolute()

# data_packs = MidvPackage.read_midv500_dataset(DATASET_PATH)
# len(data_packs), type(data_packs[0])

PosixPath('/home/krotovan/Alexander/ocr/midv500_compressed')

In [18]:
# Собираем список пакетов (MidvPackage) 
data_packs = MidvPackage.read_midv500_dataset(DATASET_PATH)
len(data_packs), type(data_packs[0])

(50, course_intro_ocr_t1.data.MidvPackage)

In [21]:
import cv2
import numpy as np
from tqdm import tqdm

def process_image(image):
    gray_image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray_image, None)
    return gray_image, keypoints, descriptors
# метод RANSAC
def find_homography(template_keypoints, target_keypoints, matches, reproj_thresh=5.0):
    src_points = np.float32([template_keypoints[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_points = np.float32([target_keypoints[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    homography_matrix, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC, reproj_thresh)
    return homography_matrix, mask

def transform_corners(template_image, homography_matrix, target_image):
    h, w = template_image.shape
    corners = np.float32([[0, 0], [w, 0], [w, h], [0, h]])
    transformed_corners = cv2.perspectiveTransform(corners[None, :, :], homography_matrix)[0]
    target_h, target_w = target_image.shape
    normalized_corners = transformed_corners / [target_w, target_h]
    return normalized_corners.tolist()

def filter_matches(matches, ratio=0.75):
    good_matches = [m for m, n in matches if m.distance < ratio * n.distance]
    return good_matches

results = {}

for package in tqdm(data_packs):
    for item in package:
        if item.is_test_split():
            try:
                template_gray, template_keypoints, template_descriptors = process_image(package.template_item.image)
                target_gray, target_keypoints, target_descriptors = process_image(item.image)

                bf_matcher = cv2.BFMatcher()
                raw_matches = bf_matcher.knnMatch(template_descriptors, target_descriptors, k=2)
                matches = filter_matches(raw_matches)

                homography_matrix, mask = find_homography(template_keypoints, target_keypoints, matches)
                normalized_corners = transform_corners(template_gray, homography_matrix, target_gray)
                results[item.unique_key] = normalized_corners
            except Exception as e:
                pass

100%|███████████████████████████████████████████| 50/50 [10:09<00:00, 12.20s/it]


In [22]:
from course_intro_ocr_t1.metrics import dump_results_dict, measure_crop_accuracy

dump_results_dict(results, Path() / 'pred.json')
accuracy = measure_crop_accuracy(
    Path() / 'pred.json',
    Path() / 'gt.json'
)
print(f'Crop accuracy: {accuracy:.4f}')

Crop accuracy: 0.9640
